<a href="https://www.kaggle.com/code/leonanvasconcelos/image-feature-extractor?scriptVersionId=163481169" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Image Feature Extractor

Extracting and displaying image features such as texture, contrast, hue, saturation, color, and homogeneity 
involves using specific image processing techniques and libraries like OpenCV and scikit-image. <br/>
Here's a Jupyter notebook in Python that demonstrates how to extract some of these features and display them for human interpretation. <br/>
This processing is meant to be a starting point for feature extraction and analysis.<br/>
<br/>
<br/>
**Author:** Leonan Vasconcelos<br/>
19/feb/2024

In [ ]:
# Kaggle specific code: takes a random file from a random Dataset
import os
import random

# Select a random dataset
kaggle_path = '/kaggle/input/'
directories = [file for file in os.listdir(kaggle_path) if os.path.isdir(kaggle_path)]
random_dataset = random.choice(directories)
directory = os.path.join(kaggle_path, random_dataset) 
print(f'Selected dataset: {random_dataset}')


# Select a random file
files = [file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]
random_file = random.choice(files)
path_image = os.path.join(directory, random_file)
print(f'Selected image: {path_image}')

# Requirements

In [ ]:
from skimage.feature import graycomatrix, graycoprops
from IPython.display import Image
import numpy as np
import cv2
from PIL import Image as PILImage
import matplotlib
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

# Utils

Below are support functions for better presentation of results.
They don't affect the analisys itself.

In [ ]:
def resize_image(image, max_resolution=1200, downscale_factor=0):
    """
    Resizes an image while maintaining the aspect ratio.

    Args:
        image: The image to be resized.
        max_resolution: The maximum size for the longest dimension of the image.
        downscale_factor: Factor by which the image dimensions are divided.

    Returns:
        The resized image and the scaling factor.
    """
    height, width = image.shape[:2]

    if downscale_factor > 0:
        scaling_factor = 1 / downscale_factor
    elif max_resolution > 0:
        scaling_factor = max_resolution / max(height, width)
    else:
        scaling_factor = 1

    new_height, new_width = int(height * scaling_factor), int(width * scaling_factor)

    # Resize and return the image along with the scaling factor
    # INTER_LANCZOS4 is the method that better preservers the texture
    return cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4), scaling_factor

def show_images(*images, titles=[], figsize=8, max_resolution=400):
    """
    Displays images in a single row of subplots with optional titles and labels reflecting the original image size.

    Args:
        *images: A sequence of images to display. Can be either numpy arrays or string paths to the images.
        titles: A list of titles for each image. Default is None.
        figsize: Size of the figure for displaying the images.
        max_resolution: The maximum resolution to be displayed.

    Returns:
        None
    """
    plt.figure(figsize=(figsize * len(images), figsize))

    for i, name_or_image in enumerate(images):
        # If the input is a string, assume it's a path to an image
        if isinstance(name_or_image, str):
            image = cv2.imread(name_or_image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = np.array(image)
        else:
            image = name_or_image

        height, width = image.shape[:2]

        # Skip images with invalid dimensions
        if height == 0 or width == 0:
            continue

        # Convert shape (3, 4, 1) to (3, 4)
        if image.ndim == 3:
            channels = image.shape[2]
            if channels == 1:
                image = np.squeeze(image, axis=-1)

        # If the image is grayscale, convert it to RGB
        if image.ndim == 2:
            image = np.stack((image,) * 3, axis=-1)

        # Resize the image if it exceeds the maximum resolution
        image, scaling_factor = resize_image(image, max_resolution)

        # Create a subplot for the image
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(image)

        # Set the title of the subplot
        title = titles[i] if titles else f"Image {i + 1}"
        ax.set_title(f"{title}", fontsize=20) 
        plt.text(0.5, -0.1, f"{width}x{height}px", ha='center', va='top', transform=plt.gca().transAxes)

        # Update tick labels to reflect original image dimensions
        def format_func_x(value, tick_number, scaling_factor=scaling_factor):
            return f"{int(value / scaling_factor)}"
        def format_func_y(value, tick_number, scaling_factor=scaling_factor):
            return f"{int(value / scaling_factor)}"

        # Adjusting the tick labels to reflect the original image size
        ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_func_x))
        ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_func_y))


    plt.tight_layout()
    plt.show()


# Example usage:
# Assuming `image_BGR` and `image_RGB` are numpy arrays of images or string paths to the images
# show_images(image_BGR, image_RGB, titles=["BGR Image", "RGB Image"])

# Image Analisys

Read the image from disk.
The image has been created with DALL-E with prompt:

"Image capturing the serene beauty of an idyllic scene in Africa, featuring a majestic white elephant near the edge of a blue lake, surrounded by lush trees with green leaves under a bright yellow sun in a clear sky."

In [ ]:
image_BGR = cv2.imread(path_image)
print(f'Image: {path_image}')
print(f'Size of image: {image_BGR.shape}')

Convert an image from BGR (Blue, Green, Red) color space to RGB (Red, Green, Blue).<br/>
This conversion is necessary because OpenCV loads images in BGR mode by default,<br/>
but for most image processing and visualization tasks, the RGB format is preferred.


In [ ]:
image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
image_RGB_original = image_RGB.copy() #save a backup of the original image

show_images(image_BGR, 
            image_RGB, 
            titles=["BGR", "RGB"], 
            max_resolution=800)   


Normalize the size of the image to not affect the results. 
When image is too large, the processing of subsequent image processing tasks is compromised.
When it is too small, details may not be noticed.

In [ ]:
image_RGB, scaling_factor = resize_image(image_RGB, max_resolution=2000)
show_images(image_RGB_original, 
            image_RGB, 
            titles=["original size", 
                    f"scale to {scaling_factor*100:.0f}%"], 
            max_resolution=800)   

## Color space conversions
First step is to convert the image to color space HSV: hue, saturation, value. <br/>
The hue channel is the most important for detect objects with distinct colors. <br/>
The saturation channel is useful to separate colorful objects.  <br/>
The value channel refer to the brightness of the object.<br/>

## HSV
<img src="https://miro.medium.com/v2/resize:fit:786/format:webp/1*qTdRziMFVUMBhKb11cm0qA.png" style="clip-path: inset(0 50% 0 0);">


In [ ]:
image_HSV = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2HSV)
show_images(image_HSV[:,:,0], image_HSV[:,:,1], image_HSV[:,:,2], titles=["Channel 0: HUE", "Channel 1: Saturation", "Channel 2: Value"])

## Grayscale and CLAHE

<p>
The V channel (Value) of the HSV (Hue, Saturation, Value) color space represents the brightness of a color, which makes it a good approximation for grayscale because it captures the luminance of the image. This is in contrast to converting an image to grayscale using the RGB (Red, Green, Blue) color channels, which involves a different approach that might not always capture perceived brightness accurately.

<p>
**Perceptual Luminance Representation:**
<br/>
The V channel in HSV is designed to represent the brightness or intensity of an image, closely aligning with human perception of brightness. This makes it a good candidate for representing an image in grayscale, as it captures the perceptual importance of different colors' luminance.

<p>
<br/>
**Color Saturation and Hue Independence:**
<br/>
Since the V channel is separate from hue and saturation, it provides a pure measure of brightness, unaffected by the color's hue or saturation. This separation can be advantageous when the color information is less important than the brightness levels for analysis or processing tasks.

<p>
<br/>
**CLAHE**
Contrast Limited Adaptive Histogram Equalization (CLAHE) is an advanced method used in computer vision to improve the contrast of images. It is an extension of adaptive histogram equalization (AHE), specifically designed to prevent the overamplification of noise that AHE tends to cause in homogeneous areas of an image. CLAHE operates by improving the local contrast and bringing out details in images, which can significantly aid various computer vision tasks. 

The improved local contrast and detail enhancement provided by CLAHE can lead to more accurate and reliable feature detection. Features such as edges, corners, and textures become more pronounced, making it easier for computer vision algorithms (like object detection, edge detection, and feature matching) to perform their tasks effectively.

In [ ]:
#extract grayscale from HS(V)
image_HSV = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2HSV)
image_GRAY = image_HSV[:,:,2]

#apply CLAHE to the image
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(18, 18))
image_GRAY_clahe = clahe.apply(image_GRAY)

show_images(image_GRAY, image_GRAY_clahe, titles=["Original", "CLAHE"])

## LAB (CIELAB)

<img src='https://sensing.konicaminolta.us/wp-content/uploads/ColorSphere.jpg'>
<br/>
Source: https://sensing.konicaminolta.us/mx/blog/entendiendo-el-espacio-de-color-cie-lab/


In [ ]:
image_LAB = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2LAB)

show_images(image_LAB[:,:,0], image_LAB[:,:,1], image_LAB[:,:,2], titles=["Channel 0: L (Lightness)", "Channel 1: A (green–red color)", "Channel 2: B (blue–yellow color)"])

It is possible to filter pixels based on its color. <br/>
In this case we will use the second channel (B)<br/>
to distinguish blue from yellow color.<br/>

In [ ]:
# In LAB, negative values means that color is blue, and positive values means that color is yellow,
# However, the cv2 returned the values as uint8 (unsigned), so we need to convert them to int8.
image_LAB_blue = image_LAB[:,:,2].astype(float)-128

# Then we apply the threshold to the image
image_LAB_blue = np.where(image_LAB_blue < 0, 255, 0).astype(np.uint8)

show_images(image_RGB, image_LAB_blue, 
            titles=["Image with Original colors", "Blue part of the image"])

The same can be used with channel A (green–red color).
However, many variances of blue contains green,
so we may need a lower threshold like -20 to select leafs.

In [ ]:
# In LAB, negative values means that color is green, and positive values means that color is red,
# However, the cv2 returned the values as uint8 (unsigned), so we need to convert them to int8.
image_LAB_green = image_LAB[:,:,1].astype(float)-128

# Then we apply the threshold to the image
image_LAB_with_green = np.where(image_LAB_green < 0, 255, 0).astype(np.uint8)
image_LAB_with_more_green = np.where(image_LAB_green < -20, 255, 0).astype(np.uint8)

show_images(image_RGB, image_LAB_with_green, image_LAB_with_more_green, 
            titles=["Image with Original colors", "Green part of the image", "Part of the image with more green"])

# Fast-GLCM

## fast_glcm functions
https://github.com/tzm030329/GLCM
Modified by LeonanUCM

In [ ]:
# coding: utf-8
# https://github.com/tzm030329/GLCM
# Modified by LeonanUCM

import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data


def main():
    pass


def fast_glcm(img, vmin=0, vmax=255, levels=8, kernel_size=5, distance=1.0, angle=0.0, kernel_shape="square"):
    '''
    Parameters
    ----------
    img: array_like, shape=(h,w), dtype=np.uint8
        input image
    vmin: int
        minimum value of input image
    vmax: int
        maximum value of input image
    levels: int
        number of grey-levels of GLCM
    kernel_size: int
        Patch size to calculate GLCM around the target pixel
    distance: float
        pixel pair distance offsets [pixel] (1.0, 2.0, and etc.)
    angle: float
        pixel pair angles [degree] (0.0, 30.0, 45.0, 90.0, and etc.)

    Returns
    -------
    Grey-level co-occurrence matrix for each pixels
    shape = (levels, levels, h, w)
    '''

    mi, ma = vmin, vmax
    ks = kernel_size
    h,w = img.shape

    # digitize
    bins = np.linspace(mi, ma+1, levels+1)
    gl1 = np.digitize(img, bins) - 1

    # make shifted image
    dx = distance*np.cos(np.deg2rad(angle))
    dy = distance*np.sin(np.deg2rad(-angle))
    mat = np.array([[1.0,0.0,-dx], [0.0,1.0,-dy]], dtype=np.float32)
    gl2 = cv2.warpAffine(gl1, mat, (w,h), flags=cv2.INTER_NEAREST,
                         borderMode=cv2.BORDER_REPLICATE)

    # make glcm
    glcm = np.zeros((levels, levels, h, w), dtype=np.uint8)
    for i in range(levels):
        for j in range(levels):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1

    if kernel_shape == "square":
        kernel = np.ones((ks, ks), dtype=np.uint8)
    else: #circle
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ks, ks))

    for i in range(levels):
        for j in range(levels):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)

    glcm = glcm.astype(np.float32)
    return glcm


def fast_glcm_mean(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm mean
    '''
    h,w = img.shape
    
    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    mean = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            mean += glcm[i,j] * i / (levels)**2

    return mean


def fast_glcm_std(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm std
    '''
    h,w = img.shape

    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    mean = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            mean += glcm[i,j] * i / (levels)**2

    std2 = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            std2 += (glcm[i,j] * i - mean)**2

    std = np.sqrt(std2)
    return std


def fast_glcm_contrast(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm contrast
    '''

    h,w = img.shape
    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    cont = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            cont += glcm[i,j] * (i-j)**2

    
    return cont


def fast_glcm_dissimilarity(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm dissimilarity
    '''
    h,w = img.shape

    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    diss = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            diss += glcm[i,j] * np.abs(i-j)
    
    return diss


def fast_glcm_homogeneity(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm homogeneity
    '''
    h,w = img.shape

    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    homo = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            homo += glcm[i,j] / (1.+(i-j)**2)

    return homo


def fast_glcm_ASM(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm asm, energy
    '''
    h,w = img.shape

    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    asm = np.zeros((h,w), dtype=np.float32)
    for i in range(levels):
        for j in range(levels):
            asm  += glcm[i,j]**2

    ene = np.sqrt(asm)
    return asm#, ene


def fast_glcm_max(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm max
    '''

    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)
        
    max_  = np.max(glcm, axis=(0,1))
    return max_


def fast_glcm_entropy(img, vmin=0, vmax=255, levels=8, ks=5, distance=1.0, angle=0.0, precalculated_glcm=None):
    '''
    calc glcm entropy
    '''

    if precalculated_glcm: 
        glcm, vmin, vmax, levels, ks, distance, angle = precalculated_glcm
    else:
        glcm = fast_glcm(img, vmin, vmax, levels, ks, distance, angle)

    pnorm = glcm / np.sum(glcm, axis=(0,1)) + 1./ks**2
    ent  = np.sum(-pnorm * np.log(pnorm), axis=(0,1))
    return ent

## Test GLCM

In [ ]:
levels = 3
kernel_size = 10
vmin, vmax = 0, 255
distance = 1.0
angle = 0.0

print(f'Calculating base GLCM matrix...')
precalculated_glcm=(fast_glcm(image_GRAY, 
                              vmin, vmax, levels, kernel_size, distance, angle), 
                              vmin, vmax, levels, kernel_size, distance, angle)

Each GLCM filter is used to highlight specific features, for example most homogeneous part of image: fast_glcm_homogeneity.
The method OTSU is used to select the threshold that better splits images into two clases based on filter aplied.

In [ ]:
# First create a list with each GLCM method available (functions)
methods = [['Contrast',      fast_glcm_contrast], 
           ['Dissimilarity', fast_glcm_dissimilarity],
           ['Homogeneity',   fast_glcm_homogeneity],
           ['Entropy',       fast_glcm_entropy],
           ['ASM',           fast_glcm_ASM], 
           ['Mean',          fast_glcm_mean], 
           ['Std',           fast_glcm_std], 
           ['Max',           fast_glcm_max]] 

for method_glcm, function_glcm in methods:
    # Calculates each GLCM method
    result_glcm = function_glcm(image_GRAY_clahe, precalculated_glcm=precalculated_glcm)
    
    # Normaliza datos from 0-255
    min_glcm, max_glcm = result_glcm.min(), result_glcm.max()
    if min_glcm != max_glcm:
        result_glcm_normal = ((result_glcm - min_glcm) / (max_glcm - min_glcm)*255).astype(np.uint8)
        threshold = threshold_otsu(result_glcm)
        threshold_pctg = (threshold-min_glcm)/(max_glcm-min_glcm)*100
    else:
        # Avoid division by zero
        result_glcm_normal = result_glcm * 0
        threshold, threshold_pctg = 0, 0
    
    print(f'\nMethod={method_glcm}:   Min={min_glcm}   Max={max_glcm}   Threshold={threshold:.3f} ({threshold_pctg:.1f}%)')
        
    # Binarize the result based on optimal division by OTSU
    mask_binary = np.where(result_glcm > threshold, 
                           255, 
                           0).astype(np.uint8)
    
    # Apply the mask to the original image to obtain the ROI
    image_selected = cv2.bitwise_and(image_RGB, image_RGB, mask=~mask_binary)

    # Display result
    show_images(result_glcm_normal, mask_binary, image_RGB, image_selected,
                titles=["Normalized Mask", "Binary Mask", "Original Image", "Selected Parts"])    